In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier


In [19]:
# load from csvs
df_features = pd.read_csv('../new_output/features.csv',index_col=0)
#df_regional = pd.read_csv('../csvs/regional_features.csv',index_col=0)

# concatenate the two df's together
#df_regional.drop(['file','label'],axis=1,inplace=True)
#df = pd.concat([df_features, df_regional], axis=1)

# drop values with null labels (blind testing data)
df_dropna = df_features.dropna()

In [20]:
X = df_dropna.drop(['file','label'],axis=1).values
#tex = np.nan_to_num(np.load('../csvs/texture_flat_1.npy'))[:1005]
#X = np.append(X,tex,axis=1)
y = df_dropna.label.values

print('X (data)',X.shape,type(X))
print('y (target)',y.shape,type(y),np.unique(y))

X (data) (1005, 38) <class 'numpy.ndarray'>
y (target) (1005,) <class 'numpy.ndarray'> [0. 1. 2. 3.]


In [82]:
X = np.load('../new_output/simple.npy')

In [83]:
X = X[:1005,:]
X = preprocessing.scale(X)

In [77]:
def getAcc(y_true,y_pred):
    return accuracy_score(y_true,y_pred)

def getPD(y_true,y_pred):
    '''get percent detected (# targets detected / # targets)'''
    return recall_score(y_true,y_pred)

def getPFA(y_true,y_pred):
    '''get percent false alarms (# false alarms / # non-targets)'''
    return 1-accuracy_score(1-y_true,1-y_pred)

In [86]:
clf = AdaBoostClassifier(n_estimators=100)
kf = KFold(n_splits=5)
PD,PD1,PD2,PD3 = [],[],[],[]
PFA = []
acc=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    predict = clf.fit(X_train, y_train!=0).predict(X_test)
    
    acc.append(getAcc(y_test!=0, predict))
    PD.append(getPD(y_test!=0,   predict))
    PD1.append(getPD(y_test==1,  predict))
    PD2.append(getPD(y_test==2,  predict))
    PD3.append(getPD(y_test==3,  predict))
    PFA.append(getPFA(y_test!=0, predict))
print('accuracy               ',np.mean(acc),'\n')
print('percent detected       ',np.mean(PD),'\n')
print('percent false alarm    ',np.mean(PFA),'\n')
print('percent detected saline',np.mean(PD1))
print('percent detected rubber',np.mean(PD2))
print('percent detected clay  ',np.mean(PD3))

accuracy                0.8248756218905472 

percent detected        0.6486097927308886 

percent false alarm     0.17512437810945275 

percent detected saline 0.5411190476190476
percent detected rubber 0.7334486961821427
percent detected clay   0.6569078144078143
